In [1]:
import h5py
import iris
import numpy as np
import xarray as xr
from pathlib import Path
from tqdm import tqdm_notebook as tqdm

From GFED README: 'The emissions group contains, for 12 months (01,02,..,12), 5 datasets: emissions (carbon with units of g C m-2 month-1 and DM with units of kg DM m-2 month-1), small_fire_fraction (unitless), daily fraction (unitless), diurnal cycle (unitless), as well as a group partitioning which contains, for both C and DM, the datasets (all unitless):
- SAVA (Savanna, grassland, and shrubland fires)
- BORF (Boreal forest fires)
- TEMF (Temperate forest fires)
- DEFO (Tropical forest fires [deforestation and degradation])
- PEAT (Peatland fires)
- AGRI (Agricultural waste burning)'

In [6]:
# Extract GFED dry matter emissions and source partitioning
path_to_gfed = Path('../../external/GFED')
group_name = 'emissions'
main_var = 'DM'
auxi_vars = ['AGRI', 'BORF', 'DEFO', 'PEAT', 'SAVA', 'TEMF', ]

monthly_dict = {}
monthly_dict[main_var.lower()] = []
for auxi_var in auxi_vars:
    monthly_dict[f'{main_var}_{auxi_var}'.lower()] = []

for fpath in tqdm(sorted(path_to_gfed.glob('*hdf5')), desc='files'):
    arr_dict = {}
    arr_dict[main_var.lower()] = []
    for auxi_var in auxi_vars:
        arr_dict[f'{main_var}_{auxi_var}'.lower()] = []

    with h5py.File(fpath, 'r') as f:
        lons = f['lon'].value[0, :]
        lats = f['lat'].value[:, 0]
        grid_cell_area = f['/ancill/grid_cell_area'].value
        xr_coord_dims = dict(coords=dict(lon=lons, lat=lats), dims=('lat', 'lon'))

        for month in f[group_name].keys():
            # Read dry matter monthly emissions
            data = f[group_name][month][main_var].value
            arr_dict[main_var.lower()].append(xr.DataArray(data, **xr_coord_dims))
            # Read monthly data on the fraction the specific source contributes to dry matter emissions
            for auxi_var in auxi_vars:
                data = f[group_name][month]['partitioning'][f'{main_var}_{auxi_var}'].value # , chunks=chunks)
                arr_dict[f'{main_var}_{auxi_var}'.lower()].append(xr.DataArray(data, **xr_coord_dims))
        for key, val in arr_dict.items():
            monthly_dict[key].append(xr.concat(val, dim='month'))

    yearly_dict = {}
    for key, val in monthly_dict.items():
        yearly_dict[key] = xr.concat(val, dim='year')
        
# Combine all data into an xarray dataset
gfed = xr.Dataset(yearly_dict)
gfed['month'] = np.arange(1,13,1)
gfed['year'] = np.arange(1997, 2017, 1)

Next cell calculates the mean over 8 GB worth of data, takes time.

In [7]:
# Calculate 20-year average mean dry matter emissions for each month
gfed_multiyear_monthly_mean = gfed.mean(dim='year')
# Rename coordinates as iris expects
gfed_multiyear_monthly_mean = gfed_multiyear_monthly_mean.rename({'month':'time', 'lat':'latitude', 'lon':'longitude'})

In [4]:
gfed_multiyear_monthly_mean.dm.attrs['long_name'] = 'dry matter emissions'
gfed_multiyear_monthly_mean.dm.attrs['units'] = 'kg/m2/month'
gfed_multiyear_monthly_mean.dm_agri.attrs['long_name'] = 'fractional contribution of agricultural waste burning to dry matter emissions'
gfed_multiyear_monthly_mean.dm_borf.attrs['long_name'] = 'fractional contribution of boreal forest fires to dry matter emissions'
gfed_multiyear_monthly_mean.dm_defo.attrs['long_name'] = 'fractional contribution of tropical forest fires to dry matter emissions'
gfed_multiyear_monthly_mean.dm_peat.attrs['long_name'] = 'fractional contribution of peatland fires to dry matter emissions'
gfed_multiyear_monthly_mean.dm_sava.attrs['long_name'] = 'fractional contribution of savanna, grassland and shrubland fires to dry matter emissions'
gfed_multiyear_monthly_mean.dm_temf.attrs['long_name'] = 'fractional contribution of temperate forest fires to dry matter emissions'
gfed_multiyear_monthly_mean.dm_agri.attrs['units'] = '1'
gfed_multiyear_monthly_mean.dm_borf.attrs['units'] = '1'
gfed_multiyear_monthly_mean.dm_defo.attrs['units'] = '1'
gfed_multiyear_monthly_mean.dm_peat.attrs['units'] = '1'
gfed_multiyear_monthly_mean.dm_sava.attrs['units'] = '1'
gfed_multiyear_monthly_mean.dm_temf.attrs['units'] = '1'
gfed_multiyear_monthly_mean['grid_cell_area'] = (('latitude', 'longitude'), grid_cell_area)
gfed_multiyear_monthly_mean.grid_cell_area.attrs['long_name'] = 'grid cell area'
gfed_multiyear_monthly_mean.grid_cell_area.attrs['units'] = 'm2'

In [5]:
# Save to .nc
gfed_multiyear_monthly_mean.to_netcdf(path=path_to_gfed / 'GFED4.1s_1997-2016_multiyear_monthly_mean_dm_emissions.nc', mode='w', format='NETCDF4')